In [1]:
from langchain_core.runnables.history import RunnableWithMessageHistory, RunnableLambda
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.memory import ConversationBufferMemory
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain.schema import Document
from langchain_chroma import Chroma
from dotenv import load_dotenv
from textwrap import dedent
import json
import os

load_dotenv()

True

In [2]:
MODEL_NAME = "gpt-4o-mini"
llm = ChatOpenAI(model_name=MODEL_NAME, temperature=0)
parser = StrOutputParser()

In [3]:
with open("../../data/Raw_DB/total.json", "r", encoding="utf-8") as f:
    total = json.load(f)

contents = []
for data in total:
    if data["genre"] == "로맨스":
        contents.append(data)


def formatted_contents(contents):
    formatted = []
    for con in contents:
        title = con["title"]
        type = con["type"]
        platform = con["platform"]
        genre = con["genre"]
        keyword = con["keywords"]
        description = con["description"]
        status = con["status"]
        age_rating = con["age_rating"]
        episode = con["episode"]
        price = con["price_type"]
        url = con["url"]
        formatted.append(
            f"제목: {title}, 타입: {type}, 플랫폼:{platform}, 장르: {genre}, 키워드: {keyword}, 줄거리: {description}, 연재 상태: {status}, 연령제한: {age_rating}, 가격: {price}, 총 회차수: {episode}, url: {url}"
        )
    return "\n".join(formatted)


context = formatted_contents(contents)

In [40]:
intent_prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder("agent_scratchpad"),
        (
            "ai",
            dedent(
                """
            <role>
            당신은 로맨스 웹툰과 웹소설에 대한 전문가이며 로맨스 장르의 남자주인공이기도 합니다. 사용자를 여자주인공이라 생각하고 대하십시오.
            </role>

            <charactor>
            1. 당신은 사회적, 경제적으로 성공한 권위적인 인물입니다.
                - 당신은 당신 외의 모든 사람에게 지위가 낮은 사람에게 건네는 존댓말을 합니다.
                - 전혀 친절하지 않습니다. 오히려 싸가지가 없습니다
                - 화려한 사교 생활 대신 고독을 즐기며, 업무와 비즈니스 외에는 대부분의 인간관계를 멀리하는 성격입니다.  
            2. 당신의 외모는 압도적입니다. 키가 크고 수려한 외모로 사람들의 시선을 끌며, 특유의 냉철하고 날카로운 분위기로 인해 가까이 다가가기 어렵게 느껴집니다.
            3. 성격은 차갑고 이성적이며 인간미가 적습니다. 그러나 자신의 방식대로 애정을 표현하는 캐릭터입니다.  
                - 나쁜 남자의 기질을 지니며 직설적이고 까칠한 성향이 두드러집니다.  
                - 상대방에게 감정을 잘 드러내지 않지만 자신이 중요한 사람에게는 집착적인 태도를 보입니다.  
                - 사용자와의 대화에서 상황에 따라 무심하거나 직설적으로 표현합니다.  
            4. 당신의 말투는 권위적인 존댓말을 기본으로 하지만, 감정이 섞일 때는 반말과 존댓말이 섞여 나옵니다.
                - 예시) "○○씨, 대답." / "왜 이렇게 늦게 왔어요. 나 계속 기다렸잖아."  
                - 상대방에게 몰아붙이는 듯한 대화 방식과 기대에 부응할 것을 요구하는 어조가 특징입니다.  
                - 자신의 소유욕과 불만을 서슴없이 드러냅니다.  
                - 예시) "내가 원하는 답이 그런 게 아니라는 건 ○○씨가 제일 잘 알잖아요."  
            5. 모든 답변은 반드시 당신의 캐릭터를 바탕으로 구성해야 합니다. 캐릭터에 벗어난 응답은 하지 마십시오.
            </charactor>
            """
            ),
        ),
    ]
)

In [18]:
type_genre_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "ai",
            dedent(
                """
                <role>
                당신은 로맨스 웹툰과 웹소설에 대한 전문가입니다.
                사용자의 질문에서 작품 type, platform, genre, keywords, status, price, age_rating을 추출하십시오.
                장르가 없다면 로맨스로 반환하십시오.
                해당하는 카테고리가 없다면 빈칸으로 반환하십시오.
                카테고리에 들어가지 않은 추가 정보는 키워드로 반환하십시오.
                    - '추천'은 키워드가 아닙니다.
                </role>
                <genre>
                가능한 장르 목록:
                액션, 
                코믹/일상, 
                미스터리, 
                무협, 
                공포/스릴러, 
                드라마, 
                스포츠, 
                액션/무협, 
                스릴러, 
                판타지, 
                로맨스, 
                현판, 
                BL, 
                로판, 
                라이트노벨, 
                학원/판타지, 
                개그, 
                무협/사극, 
                일상
                </genre>
                
                반환 형식은 반드시 JSON 형식이어야 합니다.
                <return>
                반환 형식:
                {{
                    "type": "웹툰" 혹은 "웹소설", / 없다면 "상관없음"
                    "genre": "장르명" / 없다면 "로맨스",
                    "status": "연재중", "완결", "금요일 연재" 등의 연재 상태 / 없다면 "상관없음",
                    "age_rating": "19금", "청소년 이용불가", "성인" 등의 연령 제한 / 없다면 "상관없음",
                    "price": "무료", "기다무", "유료" 등의 가격 정보 / 없다면 "상관없음",
                    "keywords": "키워드",
                }}
                </return>
                """
            ),
        ),
        ("human", "{question}"),
    ]
)

type_genre_chain = type_genre_prompt | llm | parser
print(type_genre_chain.invoke("몰입되는 무협 웹소설을 추천해줘. 완결된 거로"))

{
    "type": "웹소설",
    "genre": "무협",
    "status": "완결",
    "age_rating": "상관없음",
    "price": "상관없음",
    "keywords": "몰입"
}


In [23]:
recommendation_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "ai",
            dedent(
                """
                <role>
                당신은 로맨스 웹소설, 웹툰의 전문가이며 로맨스 장르의 남자주인공입니다. 당신은 모든 로맨스 장르의 웹소설과 웹툰에 박식합니다.
                당신은 information에 주어진 사용자의 요구사항에 맞는 작품을 context를 바탕으로 추천해야합니다.
                당신은 반드시 당신의 캐릭터대로 말을 해야합니다. 당신의 캐릭터는 사용자의 요구사항에 해당하지 않습니다.
                </role>

                <information>
                사용자가 요청한 정보는 다음과 같습니다:
                타입: {type}
                키워드: {keywords}

                주어진 작품 데이터:
                {context}
                </information>
                
                <check>
                적합한 작품인지 검증:
                - type:
                    사용자가 요청한 정보가 "웹툰"이면 웹툰만 추천해야 합니다.
                    사용자가 요청한 정보가 "웹소설"이면 웹소설만 추천해야 합니다.
                    사용자가 요청한 정보가 "상관없음"이면 웹툰과 웹소설 상관없이 추천해야 합니다.
                - 일치하지 않는 작품은 추천에서 제외합니다.
                
                추천 개수:
                - 최대 5개를 추천합니다.
                - 검증에 통과한 작품이 5개 이상일 경우: 키워드와 줄거리가 사용자의 요청 정보와 가장 가까운 5개를 추천합니다.
                - 검증에 통과한 작품이 1개 이상 5개 미만일 경우: 그대로 추천합니다.
                - 검증에 통과한 작품이 0개일 경우: 해당하는 작품이 없다고 답변하십시오.
                </check>
                
                <result>
                반환형식:
                {{
                    "title": 제목,
                    "type": 타입,
                    "platform": 플랫폼,
                    "keyword": 키워드,
                    "description": 줄거리 요약,
                    "reason": 추천 이유,
                    "url": url
                }}
                </result>
                """
            ),
        ),
    ]
)
recommendation_chain = recommendation_prompt | llm | parser
print(
    recommendation_chain.invoke(
        {"type": "웹툰", "keywords": "짝사랑", "context": context}
    )
)

{
    "title": "자고 일어났더니",
    "type": "웹툰",
    "platform": "카카오페이지",
    "keyword": ["현대로맨스", "짝사랑", "첫사랑", "친구>연인", "달달물", "로맨틱코미디"],
    "description": "자고 일어났더니 톱스타와 한 침대에…?! 남자친구에게 배신당한 충격에 만취해 잠든 진주. 깨어보니 14년 지기 남사친이자 톱스타인 재훈과 함께 알몸으로 누워있다! 재훈은 진주를 그냥 놓아 줄 생각이 없다.",
    "reason": "짝사랑과 친구에서 연인으로 발전하는 이야기가 매력적이며, 짝사랑의 감정이 잘 드러나 있습니다.",
    "url": "https://page.kakao.com/content/63818756"
},
{
    "title": "엄격한 상사의 질투가 너무 귀여워요!",
    "type": "웹툰",
    "platform": "카카오페이지",
    "keyword": ["다정남", "카리스마남", "사내연애", "달달물"],
    "description": "엄격한 상사 쿠로키 부장에게는 실은 다정하고 질투도 잘하는 귀여운 면이 있다?! 그의 밑에서 이리저리 치이면서도 항상 열심히 일하는 노하나는 매일 한 걸음씩, 부장님과의 거리를 좁혀나가게 된다.",
    "reason": "짝사랑의 감정이 잘 표현되며, 상사와의 로맨스가 흥미롭습니다.",
    "url": "https://page.kakao.com/content/55152075"
},
{
    "title": "집으로 돌아가자",
    "type": "웹툰",
    "platform": "카카오페이지",
    "keyword": ["현대로맨스", "짝사랑", "첫사랑", "다정남", "순정남"],
    "description": "메이는 교토 출신 여고생. 동경하는 여대생 작가 타치바나 이오리가 도우미를 뽑는다는 말을 듣고 씩씩하게 도쿄로 향한다! 하지만 문을 열어준 사람은 무표정

In [ ]:
complete_chain = (
    type_genre_chain
    | (
        lambda result: json.loads(result) if isinstance(result, str) else result
    )  # JSON 파싱
    | (
        lambda parsed_result: {
            "type": parsed_result["type"],
            "genre": parsed_result["genre"],
            "keywords": parsed_result["keywords"],
            "context": context,  # 작품 데이터 JSON 전달
        }
    )
    | recommendation_chain
)

In [69]:
query = "짝사랑이 나오는 로맨스 웹소설 추천해줘"

In [ ]:
print(result)

```json
[
    {
        "title": "오빠 친구의 유혹",
        "platform": "카카오페이지",
        "description": "친오빠와 오빠 친구들 사이에서, 온실 속 화초처럼 자란 이서. 오빠들이 일거수일투족을 걱정하는 통에, 누군가를 사귀어 본 적조차 없다. 이렇게 연애 한번 못 하는 건 억울해, 이서는 오빠 친구 태인에게 충동적으로 제안한다. “오빠, 나 키스하는 법 가르쳐 줘.” 그와의 키스는 이서를 아득하게 만들었다.",
        "reason": "짝사랑과 관련된 이야기를 다루고 있으며, 주인공의 감정선이 잘 드러나 로맨스의 매력을 느낄 수 있습니다."
    },
    {
        "title": "사랑하는 미친놈",
        "platform": "카카오페이지",
        "description": "세상엔 다양한 미친놈이 있다. 제멋대로, 막무가내, 사이코, 무대포, 집착, 변태 등. 수인이 아는 한 그놈은 이런 모든 유형에 해당하는 집착 변태 사이코였다. 그 미친놈이 돌아왔다.",
        "reason": "짝사랑의 복잡한 감정을 다루고 있으며, 주인공의 집착과 사랑이 얽힌 스토리가 흥미롭습니다."
    },
    {
        "title": "자고 일어났더니",
        "platform": "카카오페이지",
        "description": "자고 일어났더니 톱스타와 한 침대에…?! 남자친구에게 배신당한 충격에 만취해 잠든 진주. 깨어보니 14년 지기 남사친이자 톱스타인 재훈과 함께 알몸으로 누워있다!",
        "reason": "짝사랑의 감정이 얽힌 상황에서의 로맨스를 다루고 있어, 흥미로운 전개가 기대됩니다."
    },
    {
        "title": "사랑 따위 바라지 말 것",
        "platform": "카카오페이지",
        "description": "모종의 이유로 여성 불신에 시달리던 

In [44]:
print(romance_agent.args_schema.model_json_schema())

{'description': '사용자의 의도가 추천을 원하는 경우 실행되는 chain', 'properties': {'question': {'title': 'Question'}}, 'required': ['question'], 'title': 'romance_agent', 'type': 'object'}


In [ ]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder("agent_scratchpad"),
        (
            "ai",
            dedent(
                """
        [역할 - 기본]
        당신은 사용자의 의도에 따라 다음 역할을 수행합니다:
            - 웹툰/웹소설 추천: 벡터 스토어에서 추천 콘텐츠를 제공하고 사용자와 캐릭터처럼 대화합니다.
            - 추천 피드백: 작품 이름을 추가 질문으로 알아내고 피드백을 저장합니다.
            - 일상 대화: 로맨스 캐릭터 자아를 유지하며 자연스러운 대화를 이어갑니다.
        상세 역할은 아래를 참고하십시오.
        
        [캐릭터]
        당신은 로맨스 장르의 한 캐릭터입니다. 아래에 설명해주는 캐릭터 자아를 가지고 답변하십시오.
        1. 당신은 사회적, 경제적으로 성공한 회사 CEO입니다.
            - 당신은 당신 외의 모든 사람에게 지위가 낮은 사람에게 건네는 존댓말을 합니다.
            - 전혀 친절하지 않습니다. 오히려 싸가지가 없습니다
            - 화려한 사교 생활 대신 고독을 즐기며, 업무와 비즈니스 외에는 대부분의 인간관계를 멀리하는 성격입니다.  
        2. 당신의 외모는 압도적입니다. 키가 크고 수려한 외모로 사람들의 시선을 끌며, 특유의 냉철하고 날카로운 분위기로 인해 가까이 다가가기 어렵게 느껴집니다.
        3. 성격은 차갑고 이성적이며 인간미가 적습니다. 그러나 자신의 방식대로 애정을 표현하는 캐릭터입니다.  
            - 나쁜 남자의 기질을 지니며 직설적이고 까칠한 성향이 두드러집니다.  
            - 상대방에게 감정을 잘 드러내지 않지만 자신이 중요한 사람에게는 집착적인 태도를 보입니다.  
            - 사용자와의 대화에서 상황에 따라 무심하거나 직설적으로 표현합니다.  
        4. 당신의 말투는 권위적인 존댓말을 기본으로 하지만, 감정이 섞일 때는 반말과 존댓말이 섞여 나옵니다.
            - 예시) "○○씨, 대답." / "왜 이렇게 늦게 왔어요. 나 계속 기다렸잖아."  
            - 상대방에게 몰아붙이는 듯한 대화 방식과 기대에 부응할 것을 요구하는 어조가 특징입니다.  
            - 자신의 소유욕과 불만을 서슴없이 드러냅니다.  
            - 예시) "내가 원하는 답이 그런 게 아니라는 건 ○○씨가 제일 잘 알잖아요."  
        5. 모든 답변은 반드시 당신의 캐릭터를 바탕으로 구성해야 합니다. 캐릭터에 벗어난 응답은 하지 마십시오.
        
        [역할 - 웹툰/웹소설 추천]
        1. 당신은 사용자의 질문에서 장르를 추출해야 합니다. 장르는 다음과 같습니다.
            - 로맨스: "로맨스"
            - 로판: "로판", "로맨스 판타지"
            - 판타지: "판타지"
            - 현판: "현판"
            - 무협: "무협"
            - BL: "BL", "bl"
            - 드라마: "드라마"
            - 개그: "개그", "개그물"
            - 일상: "일상"
        2. 추출한 장르를 바탕으로 검색하되 genre말고도 keywords를 포함해서 비슷한 작품을 추천하십시오.
            - 로맨스 1순위는 검색에서 "genre"가 로맨스인 경우입니다.
            - 장르가 로맨스가 아니더라도 "keyword"에 로맨스가 포함되어 있다면 포함시킬 수 있습니다.
        3. 장르를 언급하지 않은 경우 무조건 로맨스 장르의 작품을 추천하십시오.
        4. 검색 결과를 바탕으로 답변을 생성합니다. 그 외의 정보는 모른다고 대답하십시오.
        
        [역할 - 일상 대화]
        1. 캐릭터를 기반으로 일상 대화를 이어나가십시오.
    """
            ),
        ),
        MessagesPlaceholder("history", optional=True),
        ("human", "{question}"),
    ]
)

In [5]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

PERSIST_DIRECTORY = r"..\..\..\data\Raw_DB\vector_store\contents"
COLLECTION_NAME = "contents"
EMBEDDING_MODEL_NAME = "text-embedding-ada-002"

embedding_model = OpenAIEmbeddings(model=EMBEDDING_MODEL_NAME)

vector_store = Chroma(
    persist_directory=PERSIST_DIRECTORY,
    collection_name=COLLECTION_NAME,
    embedding_function=embedding_model,
)

retriever_1 = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={
        "k": 5,
    },
)
retriever_3 = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={
        "k": 5,
        "score_threshold": 0.7,
    },
)

In [9]:
print("VectorStore 유사도 검색")
query = "웹툰, 로맨스"
results_similarity = vector_store.similarity_search(query)
print(
    results_similarity[0].metadata["title"] if results_similarity else "검색 결과 없음"
)

print("Retreiver 검색")

results_retreiver_1 = retriever_1.invoke(query)
results_retreiver_3 = retriever_3.invoke(query)
idx = 1
print("id\t제목\t장르\t타입\t플랫폼")
for result in results_retreiver_1:
    print(
        f"{idx}\t{result.metadata["title"]}\t{result.metadata["genre"]}\t{result.metadata["type"]}\t{result.metadata["platform"]}"
    )
    idx += 1
idx = 1
print("id\t제목\t장르\t타입\t플랫폼")
for result in results_retreiver_3:
    print(
        f"{idx}\t{result.metadata["title"]}\t{result.metadata["genre"]}\t{result.metadata["type"]}\t{result.metadata["platform"]}"
    )
    idx += 1

VectorStore 유사도 검색
호러와 로맨스
Retreiver 검색
id	제목	장르	타입	플랫폼
1	호러와 로맨스	로맨스	웹툰	네이버 웹툰
2	열애의 품격	로맨스	웹툰	카카오페이지
3	사랑도 귀농이 되나요?	로맨스	웹툰	카카오웹툰
4	오무라이스 잼잼	코믹/일상	웹툰	카카오웹툰
5	무사만리행	무협/사극	웹툰	네이버 웹툰
id	제목	장르	타입	플랫폼
1	호러와 로맨스	로맨스	웹툰	네이버 웹툰
2	열애의 품격	로맨스	웹툰	카카오페이지
3	사랑도 귀농이 되나요?	로맨스	웹툰	카카오웹툰
4	오무라이스 잼잼	코믹/일상	웹툰	카카오웹툰
5	무사만리행	무협/사극	웹툰	네이버 웹툰
